In [1]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.0005
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.997
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=20000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(128, input_dim=self.state_size, activation='relu'),
            Dense(128, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=RMSprop(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        processed = np.array(state).astype(float)
        processed[processed == 1] = -1
        processed[processed == 2] = 1   
        return processed

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.05
    decay_episodes = 1000
    
    for episode in range(episodes):
        epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
        agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 2000) * epsilon_decay) if episode % 2000 != 0 else 1.0
        
        smartness = min(1, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.5  # Increased reward for winning
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.5  # Increased penalty for losing
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = 0.0
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model10_episode_{episode}.h5')
    agent.model.save('model10.h5')
    

agent = train_agent()

"""
Changes made: Increased reward for wins and penalty for losses to prioritize winning over drawing, reduced minimum epsilon for more exploration early on, slowed epsilon decay for a smoother transition to exploitation, increased model capacity with more neurons, and saved checkpoints every 1000 episodes to track progress. This setup aims to maximize the likelihood of achieving at least one more win than losses over 1000 games, regardless of the number of draws.
"""


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 8
Episode: 0, Win Rate: 1.00, Epsilon: 1.000, smartmove0.0
Wins: 1, Losses: 0, Draws: 0
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (S

Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 8
Episode: 1000, Win Rate: 0.44, Epsilon: 0.050, smartmove0.16666666666666666
Wins: 445, Losses: 429, Draws: 127
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) choos

Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 1
Episode: 2000, Win Rate: 0.54, Epsilon: 1.000, smartmove0.3333333333333333
Wins: 1078, Losses: 682, Draws: 241
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) choos

Episode: 3000, Win Rate: 0.47, Epsilon: 0.050, smartmove0.5
Wins: 1414, Losses: 1185, Draws: 402
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 6


...

Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 5
Episode: 6000, Win Rate: 0.35, Epsilon: 1.000, smartmove1
Wins: 2130, Losses: 2363, Draws: 1508
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 9
P

Episode: 7000, Win Rate: 0.31, Epsilon: 0.050, smartmove1
Wins: 2180, Losses: 2981, Draws: 1840
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 6
P

Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 8
Episode: 8000, Win Rate: 0.28, Epsilon: 1.000, smartmove1
Wins: 2242, Losses: 3143, Draws: 2616
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 1
P

Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 7
Episode: 9000, Win Rate: 0.25, Epsilon: 0.050, smartmove1
Wins: 2290, Losses: 3726, Draws: 2985
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 5
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
P

'\nChanges made: Increased reward for wins and penalty for losses to prioritize winning over drawing, reduced minimum epsilon for more exploration early on, slowed epsilon decay for a smoother transition to exploitation, increased model capacity with more neurons, and saved checkpoints every 1000 episodes to track progress. This setup aims to maximize the likelihood of achieving at least one more win than losses over 1000 games, regardless of the number of draws.\n'

In [6]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class PlayerSQN:
    def __init__(self, model_name=None):
        self.state_size = 9
        self.action_size = 9
        self.gamma = 0.95
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.win_count = 0
        self.loss_count = 0
        
        if model_name:
            self.model = self.load_model(model_name)
        else:
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model
    
    def load_model(self, model_name):
        from tensorflow.keras.models import load_model
        return load_model(model_name)

    def process_state(self, state):
        processed = np.array(state).astype(float)
        processed[processed == 1] = -1
        processed[processed == 2] = 1
        return processed

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        # Adjust epsilon based on win/loss ratio
        if self.win_count + self.loss_count > 0:
            win_rate = self.win_count / (self.win_count + self.loss_count)
            self.epsilon = max(self.epsilon_min, min(1.0, 1.0 - win_rate))

        if random.random() < self.epsilon:
            # Prioritize center and corners during exploration
            center = 4
            corners = [0, 2, 6, 8]
            if center in valid_actions:
                return center
            corner_options = [c for c in corners if c in valid_actions]
            if corner_options:
                return random.choice(corner_options)
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        
        # Add position bias to Q-values
        position_bias = np.zeros(9)
        position_bias[4] = 0.2  # Center bias
        for corner in [0, 2, 6, 8]:
            position_bias[corner] = 0.1  # Corner bias
        q_values += position_bias
        
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = PlayerSQN()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    best_win_ratio = 0
    best_model = None
    
    for episode in range(episodes):
        # Dynamic smartness based on performance
        smartness = min(0.8, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        
        state = game.board
        done = False
        
        while not done:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 2.0  # Increased reward for winning
                history['wins'] += 1
                agent.win_count += 1
                done = True
            elif game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.loss_count += 1
                done = True
            elif game.is_full():
                reward = 0.5  # Slight positive reward for draws
                history['draws'] += 1
                done = True
                
            next_state = game.board
            
            # Store experience and train
            agent.store_experience(state, action, reward, next_state, done)
            state = next_state
            
            if not done:
                game.player1_move()
                if game.current_winner == 1:
                    reward = -1.0
                    history['losses'] += 1
                    agent.loss_count += 1
                    done = True
                    agent.store_experience(state, action, reward, game.board, done)
            
            agent.train()
        
        # Save model every 1000 episodes
        if (episode + 1) % 1000 == 0:
            model_name = f'model11_episode_{episode + 1}.h5'
            agent.model.save(model_name)
            print(f"Model saved as {model_name}")
        
        # Track performance
        if episode % 100 == 0:
            total_games = history['wins'] + history['losses']
            win_ratio = history['wins'] / total_games if total_games > 0 else 0
            print(f"Episode: {episode}, Win Rate: {win_ratio:.2f}, Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
            
    
    # Save final model
    agent.model.save('model11.h5')
    return agent

if __name__ == "__main__":
    agent = train_agent()

Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 4
Episode: 0, Win Rate: 1.00, Wins: 1, Losses: 0, Draws: 0
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position

Player 1 (Smart/Random) chooses position 1
Model saved as model11_episode_1000.h5
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 4
Episode: 1000, Win Rate: 0.95, Wins: 925, Losses: 52, Draws: 24
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses posit

Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 4
Model saved as model11_episode_2000.h5
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 1
Episode: 2000, Win Rate: 0.93, Wins: 1821, Losses: 130, Draws: 50
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses pos

Player 1 (Smart/Random) chooses position 3
Model saved as model11_episode_3000.h5
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 7
Episode: 3000, Win Rate: 0.91, Wins: 2624, Losses: 274, Draws: 103
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses po

Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Model saved as model11_episode_4000.h5
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Episode: 4000, Win Rate: 0.86, Wins: 3312, Losses: 517, Draws: 172
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses po

...

Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 4
Model saved as model11_episode_6000.h5
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Episode: 6000, Win Rate: 0.76, Wins: 4281, Losses: 1375, Draws: 345
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses p

Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 6
Model saved as model11_episode_7000.h5
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 8
Episode: 7000, Win Rate: 0.72, Wins: 4758, Losses: 1840, Draws: 403
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses p

Player 1 (Smart/Random) chooses position 8
Model saved as model11_episode_8000.h5
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 2
Episode: 8000, Win Rate: 0.69, Wins: 5206, Losses: 2324, Draws: 471
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses p

Player 1 (Smart/Random) chooses position 6
Model saved as model11_episode_9000.h5
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 6
Episode: 9000, Win Rate: 0.67, Wins: 5621, Losses: 2823, Draws: 557
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses p

Player 1 (Smart/Random) chooses position 6
Model saved as model11_episode_10000.h5


In [ ]:
# Previous code remains the same until the train_agent function


def train_agent(episodes=10000):
    agent = PlayerSQN()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    best_draw_ratio = 0
    best_model = None
    
    for episode in range(episodes):
        # Dynamic opponent strength
        smartness = min(0.95, episode / (episodes * 0.4))  # Faster increase in difficulty
        game = TicTacToe(smartMovePlayer1=smartness)
        
        state = game.board
        done = False
        
        while not done:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                agent.draw_count += 1
                break
                
            action = agent.select_action(state, valid_actions, smartness)
            game.make_move(action, player=2)
            
            # Reward structure heavily favoring draws over risky win attempts
            reward = 0
            if game.current_winner == 2:
                reward = 1.0  # Moderate reward for winning
                history['wins'] += 1
                agent.win_count += 1
                done = True
            elif game.current_winner == 1:
                reward = -5.0  # Severe penalty for losing
                history['losses'] += 1
                agent.loss_count += 1
                done = True
            elif game.is_full():
                reward = 0.8  # High reward for achieving a draw
                history['draws'] += 1
                agent.draw_count += 1
                done = True
                
            next_state = game.board
            agent.store_experience(state, action, reward, next_state, done)
            state = next_state
            
            if not done:
                game.player1_move()
                if game.current_winner == 1:
                    reward = -5.0  # Severe penalty for losing
                    history['losses'] += 1
                    agent.loss_count += 1
                    done = True
                    agent.store_experience(state, action, reward, game.board, done)
            
            agent.train()
        
        # Save model every 1000 episodes
        if (episode + 1) % 1000 == 0:
            model_name = f'defence13_episode_{episode + 1}.h5'
            agent.model.save(model_name)
            print(f"Model saved as {model_name}")
        
        # Track performance with focus on minimizing losses
        if episode % 100 == 0:
            total_games = history['wins'] + history['losses'] + history['draws']
            if total_games > 0:
                win_ratio = history['wins'] / total_games
                loss_ratio = history['losses'] / total_games
                draw_ratio = history['draws'] / total_games
                
                print(f"Episode: {episode}")
                print(f"Win/Draw/Loss: {win_ratio:.3f}/{draw_ratio:.3f}/{loss_ratio:.3f}")
                print(f"Wins: {history['wins']}, Draws: {history['draws']}, Losses: {history['losses']}")
                print(f"Current Epsilon: {agent.epsilon:.2f}")
                print(f"Current Smartness: {smartness:.2f}\n")
                
                # Save best model based on minimizing losses while maintaining some wins
                current_score = draw_ratio + (win_ratio * 0.5) - (loss_ratio * 2)
                if current_score > best_draw_ratio and history['losses'] == 0:
                    best_draw_ratio = current_score
                    best_model = f'defence13_best.h5'
                    agent.model.save(best_model)
                    print(f"New best model saved! Score: {current_score:.3f}\n")
    
    # Save final model
    agent.model.save('defence13_final.h5')
    return agent

if __name__ == "__main__":
    agent = train_agent()

Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 8
Episode: 0
Win/Draw/Loss: 1.000/0.000/0.000
Wins: 1, Draws: 0, Losses: 0
Current Epsilon: 1.00
Current Smartness: 0.00

New best model saved! Score: 0.500

Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 1
Player 1 (Smart/Random) chooses position 6
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 2
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 7
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) chooses position 3
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 8
Player 1 (Smart/Random) chooses position 9
Player 1 (Smart/Random) chooses position 4
Player 1 (Smart/Random) cho

ValueError: Could not interpret loss identifier: huber_loss